# Downloads raw data

In [6]:
import API.send_request as req
import json

# match information
match_id = 'EUN1_2960023848'
server = 'europe'
save_path = '../../Dataset/League_of_Legends/challenger_raw/'

sender = req.match_req_sender(server)
meta = sender.req_match(match_id)
timeline = sender.req_timeline(match_id)

if (meta != None and timeline != None):
    meta = meta.json()
    timeline = timeline.json()

    # save json file
    with open(save_path+f"{match_id}.json", "w") as file_match:
        json.dump(meta, file_match, indent=4)

    with open(save_path+f"{match_id}_timeline.json", "w") as file_timeline:
        json.dump(timeline, file_timeline, indent=4)

Got match data with match id=EUN1_2960023848 successfuly.
Got timeline data with match id=EUN1_2960023848 successfuly.


# Extract features

In [25]:
import pandas as pd
from os import listdir
from tqdm import tqdm
from torch import tensor, zeros

pickle_route = "../../Dataset/League_of_Legends/features_tensor/challenger/"

print("Processing flipped train features...")
csv_route = "../../Dataset/League_of_Legends/challenger_csv/"
feat_list = listdir(csv_route)

for file_name in tqdm(feat_list):
    file = pd.read_csv(f"{csv_route}{file_name}")
    file = file.sort_values(by='time', ascending=False, axis=0, ignore_index=True)
    file = file[file['is_valid'] == True]
    file = file.loc[:,"time":"event_weight"]
    players = []
    for idx in range(0, 10):
        player = file[file["player"] == idx+1].drop(columns=["player"])
        temp_feature = tensor([player.values]) if len(player) != 0 else zeros([1, 1, 30])
        # if temp_feature.size()[1] > 1: temp_feature = temp_feature.flip(1)
        players.append(temp_feature)
    pd.to_pickle(players, f"{pickle_route}{file_name[0:-4]}.pkl")
print("")

Processing flipped train features...


100%|██████████| 12/12 [00:00<00:00, 59.27it/s]

# Test


In [26]:
import pasta_spaghettini as spa
from tqdm import tqdm
import pandas as pd

hp = {"epochs": 1, "learning_rate": 0.01, "num_layers": 1, "hidden_size": 8, "input_size": 30}
PASTA = spa.PASTA(hp["input_size"], hp["hidden_size"], hp["num_layers"], hp["learning_rate"])

route = "../../Dataset/League_of_Legends/features_tensor/challenger/"

test_match_result_ftr = pd.read_csv("./processed_csvs/challenger_result.csv")
total_rows = test_match_result_ftr.shape[0]

PASTA.load_parameter(postfix="_spaghettini")
confusion_matrix, c_label = [0, 0, 0, 0], ["TN", "FP", "FN", "TP"]

print("--test start--")
for idx, row in tqdm(test_match_result_ftr.iterrows(), ncols=50, total=total_rows):
    match_id, win = row["match_no"], row["win"]
    features = pd.read_pickle(f"{route}{match_id}.pkl")
    winner, predict = PASTA.test(features, win, match_id, postfix="challenger")
    correct = 2*winner + predict
    confusion_matrix[correct] += 1

for idx in range(0, 4):
    print(f"{c_label[idx]}: {confusion_matrix[idx]} / ", end="")

--test start--


100%|█████████████| 12/12 [00:00<00:00, 66.59it/s]

TN: 8 / FP: 0 / FN: 0 / TP: 4 / 

# Score analysis

In [ ]:
import experiment_vis_leaderboard as lb

match_id = '210510_eun1_2825546524'
# match_id = '210510_euw1_5253755356' # too many WARD_PLACED
# match_id = '210510_kr_5179842525'

data = lb.load_leaderboard(match_id)
data.drop(labels='sum of ranks', axis='columns', inplace=True)
data